In [1]:
import pandas as pd
import numpy as np
table_all_current = pd.read_pickle('table_all_current_7.29.19.pkl')
pd.__version__

'0.24.2'

In [2]:
table_all_current.shape

(67, 1110)

In [3]:
table_all_current.tail()

,prob_default,prob_fullypaid,return_preds,loan_amnt,funded_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,...,emp_title_2_Technician,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver
id,,,,,,,,,,,,,,,,,,,,,
156077320,0.101173,0.898827,0.066045,35450,7250.0,0.1033,1.0,40000.0,26.04,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155809619,0.747000,0.253000,0.089305,40000,7925.0,0.1524,8.0,102612.0,26.85,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155879545,0.006096,0.993904,0.088831,40000,6050.0,0.1430,6.0,120000.0,9.70,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156086684,0.592538,0.407462,0.178415,40000,4800.0,0.2500,1.0,135000.0,17.70,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156050759,0.021257,0.978743,0.046662,40000,4350.0,0.0756,1.0,110000.0,34.72,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def expected_portfolio_return_evenly_weighted(table_all_current, avail_funds):
    evenly_weighted_expected_returns = []
    for idx in table_all_current.index:
        evenly_weighted_expected_returns.append(
            (table_all_current['return_preds'][idx]) * (avail_funds/len(table_all_current)))
    expected_portfolio_return = sum(evenly_weighted_expected_returns)/avail_funds
    return expected_portfolio_return

def rank_table_by_shrop_ratio_RAR(table_all_current, avail_funds):
    table_all_current['shrop_ratio'] = np.nan
    for idx in table_all_current.index:
        table_all_current['shrop_ratio'][idx] = (
            (expected_portfolio_return_evenly_weighted(table_all_current, avail_funds) - 0.0188) / 
                                                        (table_all_current['prob_default'][idx]))
    table_all_current_ranked = table_all_current.sort_values(by='shrop_ratio',axis=0, ascending=False)
    return table_all_current_ranked

def recommended_loans_ranked_by_shrop_RAR(table_all_current, max_prob_default, min_desired_return, avail_funds):
    rec_table = table_all_current[(table_all_current['prob_default'] <= max_prob_default) & 
                             (table_all_current['return_preds'] >= min_desired_return)]
    rec_table_ranked = rank_table_by_shrop_ratio_RAR(rec_table, avail_funds)
    return rec_table_ranked

def portfolio_prob_default_evenly_weighted(table_all_current, avail_funds):
    evenly_weighted_prob_default = []
    for idx in table_all_current.index:
        evenly_weighted_prob_default.append(
            (table_all_current['prob_default'][idx]) * (avail_funds/len(table_all_current)))
    portfolio_prob_default = sum(evenly_weighted_prob_default)/avail_funds
    return portfolio_prob_default

def portfolio_shrop_ratio_evenly_weighted(table_all_current, avail_funds):
    table_all_current_ranked = rank_table_by_shrop_ratio_RAR(table_all_current, avail_funds)
    evenly_weighted_shrop_ratio = []
    for idx in table_all_current_ranked.index:
        evenly_weighted_shrop_ratio.append(
            (table_all_current['shrop_ratio'][idx]) * (avail_funds/len(table_all_current)))
    portfolio_shrop_ratio = sum(evenly_weighted_shrop_ratio)/avail_funds
    return portfolio_shrop_ratio

def summarize_recommendation(table_all_current, max_prob_default, min_desired_return, avail_funds):
    rec_table_ranked = recommended_loans_ranked_by_shrop_RAR(table_all_current, max_prob_default, min_desired_return, avail_funds)
    port_prob_def = portfolio_prob_default_evenly_weighted(rec_table_ranked,avail_funds)
    port_exp_return = expected_portfolio_return_evenly_weighted(rec_table_ranked, avail_funds)
    port_shrop_ratio = portfolio_shrop_ratio_evenly_weighted(rec_table_ranked,avail_funds)
    max_investable = rec_table_ranked['loan_amnt'].sum() - rec_table_ranked['funded_amnt'].sum()
    return (rec_table_ranked,port_prob_def,port_exp_return,port_shrop_ratio,max_investable)

In [5]:
(rec_table_ranked,port_prob_def,
 port_exp_return,port_shrop_ratio,max_investable) = summarize_recommendation(table_all_current,
                                                                             max_prob_default=0.2,
                                                                             min_desired_return=0.07,
                                                                             avail_funds=10000)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_

In [6]:
rec_table_ranked

,prob_default,prob_fullypaid,return_preds,loan_amnt,funded_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,...,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver,shrop_ratio
id,,,,,,,,,,,,,,,,,,,,,
155976076,0.005641,0.994359,0.093169,35000,31150.0,0.1430,4.0,96000.0,27.96,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.466940
155879545,0.006096,0.993904,0.088831,40000,6050.0,0.1430,6.0,120000.0,9.70,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.460879
155915661,0.007458,0.992542,0.078394,31150,23900.0,0.1240,6.0,155000.0,28.45,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.186148
154588667,0.008160,0.991840,0.073181,20000,10000.0,0.1102,1.0,100000.0,27.28,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.309593
156116626,0.025337,0.974663,0.179856,25000,16625.0,0.2500,6.0,100000.0,20.71,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.998229
155189447,0.027131,0.972869,0.079783,19000,5675.0,0.1695,1.0,50000.0,14.88,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.799973
155466042,0.031727,0.968273,0.095554,34025,14950.0,0.1430,1.0,45000.0,32.99,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.394384
155964253,0.044189,0.955811,0.105994,10000,5000.0,0.1524,1.0,25000.0,29.24,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.719130
155780689,0.052878,0.947122,0.113295,35000,19975.0,0.1612,3.0,160000.0,25.32,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.436629


In [7]:
port_prob_def

0.04148772358894349

In [8]:
port_exp_return

0.09476678749345081

In [9]:
port_shrop_ratio

4.93803789364291

In [10]:
max_investable

161825.0